<a href="https://colab.research.google.com/github/Kim-TaeKyoung/a/blob/main/1_2_EfficentDet_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 환경세팅(의존성 패키지 설치)

In [ ]:
import os
import sys
import random
import json
import time
import datetime
from IPython.display import Image, display

# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 패키지 설치
!cd /content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet && cat requirements_colab.txt | xargs -n 1 -L 1 pip install
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
!pip install efficientnet_pytorch
!pip install tensorboardX

sys.path.append("/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/");
from train_detector import Detector
from infer_detector import Infer

In [ ]:
# Augmentation이 적용된 데이터셋 다운로드
!curl -L "https://app.roboflow.com/ds/D6ipNcT3p5?key=dPqULorgrb" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

!mkdir Fire
!mkdir Fire/annotations
!mkdir Fire/Images
%cp train/_annotations.coco.json Fire/annotations/instances_Images.json
%cp train/*.jpg Fire/Images/

# 2. 데이터셋 및 GPU 설정

In [ ]:
# 모델선언
aics = Detector()

root_dir = "./";
coco_dir = "Fire";
img_dir = "./";
set_dir = "Images";

# 데이터셋 정의
aics.Train_Dataset(root_dir, coco_dir, img_dir, set_dir, batch_size=16, image_size=512, use_gpu=True)

GPU
1
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


# 3. 훈련

In [ ]:
def weight_reset(m):
    reset_parameters = getattr(m, "reset_parameters", None)
    if callable(reset_parameters):
        m.reset_parameters()

In [ ]:
#모델 불러오기(Efficinet-b0)
aics.Model()

#옵티마이저 셋팅, Optimizer는 Adam, learning rate는 0.01, 3 epochs 마다 loss drop이 없을경우 learning_rate는 lr * 0.1로 감소, Early Stop은 껐음
aics.Set_Hyperparams(lr=0.0001, val_interval=10, es_min_delta=0.0, es_patience=0)
aics.system_dict['local']['model'].apply(weight_reset)

In [ ]:
birthdate = datetime.datetime.now(datetime.timezone.utc)
weight_save_path = birthdate.astimezone().strftime('%Y-%m-%d %H:%M:%S %Z')
print(weight_save_path)

2021-10-22 01:05:01 UTC


In [ ]:
%%time

aics.Train(num_epochs=500, model_output_dir="/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/weight/"+ weight_save_path);

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./tensorboard/signatrix_efficientdet_coco/

In [ ]:
# Validation 이미지 결과 확인

aics = Detector()
aics = Infer();
aics.Model(model_dir="/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/weight/2021-10-19 09:45:22 UTC")

class_list = ['L','fire', 'smoke']

import glob
from PIL import Image as PILImage
from matplotlib import pyplot as plt
from IPython.display import Image, clear_output, display
from IPython.core.magic import register_line_cell_magic
from natsort import natsorted

plt.rcParams.update({'figure.max_open_warning': 0})

test_data_dir = '/content/drive/MyDrive/Colab/AI_ChampionShip/dataset/test-frame_rearranged/*.jpg'
test_images = natsorted(glob.glob(test_data_dir))


for fig in test_images:
  aics.Predict(fig, class_list, vis_threshold=0.5);
  display(Image(filename='output.jpg'))
  print("\n")
  

In [ ]:
import mAP_evaluation
import importlib
import torch

importlib.reload(mAP_evaluation)

# 모델선언
aics = Detector()

root_dir = "./";
coco_dir = "Fire_Valid";
img_dir = "./";
set_dir = "Images";

!mkdir -p Fire_Valid/annotations
!mkdir -p Fire_Valid/Images
%cp valid/_annotations.coco.json Fire_Valid/annotations/instances_Images.json
%cp valid/*.jpg Fire_Valid/Images/

aics.Val_Dataset(root_dir, coco_dir, img_dir, set_dir)

#모델 불러오기(Efficinet-b0)
model = torch.load("/content/drive/MyDrive/Colab/AI_ChampionShip/EfficientDet/weight/2021-09-30_17_02_38/signatrix_efficientdet_coco.pth")
val_cocoset = aics.system_dict["local"]["val_set"]

model.cuda()

mAP_evaluation.evaluate_coco(val_cocoset, model)